In [1]:
%matplotlib inline
import mxnet as mx
import numpy as np
import matplotlib.pyplot as plt



## vgg 微调

In [8]:
sym, arg_params, aux_params = mx.model.load_checkpoint(prefix='model/vgg16', epoch=0)

## Resnet101 微调

In [14]:
sym, arg_params, aux_params = mx.model.load_checkpoint(prefix='model/resnet-101', epoch=0)